In [130]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,f1_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [117]:
train=pd.read_csv("train_mod.csv")
test=pd.read_csv("test_mod.csv")


In [131]:
lrc = LogisticRegression()
rfc=RandomForestClassifier()
adbc=AdaBoostClassifier()
bgc=BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dtc = DecisionTreeClassifier()
bgcl_lrc = BaggingClassifier(base_estimator=lrc, random_state=42)
ab_rfc = AdaBoostClassifier(base_estimator=rfc,random_state=42)
ab_dtc = AdaBoostClassifier(base_estimator=dtc,random_state=42)
ab_nbc=  AdaBoostClassifier(base_estimator=gnb,random_state=42)
ab_lrc=  AdaBoostClassifier(base_estimator=lrc,random_state=42)
gbc=GradientBoostingClassifier()
ab_gbc=  AdaBoostClassifier(base_estimator=gbc,random_state=42)
xgbc=XGBClassifier()
ab_xgbc=  AdaBoostClassifier(base_estimator=xgbc,random_state=42)
lgbc=LGBMClassifier()
CatBoostClassifier=CatBoostClassifier()

In [119]:
train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,days_stayed,total_days_stayed,total_hours_stayed,condition_is_missing
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,73 days 16:25:00.000000000,73,16.42,0
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2,1862 days 17:47:00.000000000,1862,17.78,0
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,-1.0,Brown,0.15,40.90,15,4,2.0,4,752 days 08:24:00.000000000,752,8.40,1
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2,755 days 18:30:00.000000000,755,18.50,0
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1,52 days 09:38:00.000000000,52,9.63,0


In [120]:
#label encoding the features for the modelling process
le=LabelEncoder()
le.fit(train["color_type"])
# train["color_type"]=le.fit_transform(train["color_type"])
train["color_type"]=le.transform(train["color_type"])
test["color_type"]=le.transform(test["color_type"])

train_set,val_set=train_test_split(train,test_size=.25,random_state=42)
train_set.shape,val_set.shape

#train_set
train_data=train_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_res_breed=train_set.breed_category
train_res_pet=train_set.pet_category

#val_set
val_data=val_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed'],
                     axis=1)
val_res_breed=val_set.breed_category
val_res_pet=val_set.pet_category
train_data.shape,train_res_breed.shape,val_data.shape,val_res_breed.shape


((14125, 9), (14125,), (4709, 9), (4709,))

In [121]:
models=[rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,ab_rfc,ab_dtc,ab_nbc,ab_lrc,gbc,ab_gbc,xgbc,ab_xgbc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,y_train,X_test, y_test):
    for model in models:
        print(model.__class__.__name__)
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        y_test_pred_new=model.predict_proba(X_test)
        y_test_pred_new=y_test_pred_new[:,1]
        train_score=model.score(X_train,y_train)
        test_score=model.score(X_test,y_test)
        p_score=metrics.precision_score(y_test,y_test_pred,average='micro')
        r_score=metrics.recall_score(y_test,y_test_pred,average='micro')
        
#         ac=metrics.roc_auc_score(y_test,y_test_pred_new,multi_class="ovr",average='micro')
#         acc=accuracy_score(y_test,y_test_pred_new)
            
            
        sctr.append(train_score)
        scte.append(test_score)
        ps.append(p_score)
        rs.append(r_score)
#         auc.append(ac)
#         acc.append(acc)
    return sctr,scte,ps,rs,acc




In [122]:
#checking the performance on train and validation
# ensemble_for_train_val(train_data,train_res_breed, val_data, val_res_breed)
# # 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
# ensemble1=pd.DataFrame({'names':['Random Forest','Ada boost','Bagging',
#                                 'Naive-Bayes','KNN','Decistion Tree',
#                                 'bagged LR',"adaboost rf","adaboost dtc","adaboost naive bayes",
#                                 "adaboost logistic regression","gradient boosting trees"
#                                 ,"adaboost gbc","xgboost","adaboost xgbc"],
#                        'training':sctr,'testing':scte,'precision':ps,'recall':rs,})
# ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
# ensemble

In [123]:
ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
ensemble.to_csv("results.csv",index=False)

In [124]:
train_full=train.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_full_breed=train.breed_category
train_full_pet=train.pet_category
test_full=test.drop(['pet_id', 'issue_date', 'listing_date','days_stayed']
                          ,axis=1)

train_full.shape,test_full.shape

((18834, 9), (8072, 9))

In [136]:
# # catboost
# models=[cat]


# def pred_on_full_data(Xtrain,ytrain,ytrain2,Xtest,models):
#     for model in models:
#         print(model.__class__.__name__)
#         model.fit(Xtrain, ytrain)
#         y_test_pred_breed = model.predict(Xtest)[:,0]
#         model.fit(Xtrain, ytrain2)
#         y_test_pred2_pet=model.predict(Xtest)[:,0]
#         id=pd.Series(test["pet_id"])
#         submission=pd.DataFrame({'pet_id':id,'breed_category':y_test_pred_breed,'pet_category':y_test_pred2_pet})
# #         predictions=pd.concat([test['INCIDENT_ID'],pd.DataFrame(y_test_pred,columns=['MULTIPLE_OFFENSE'])],1)
#         a="submission_multiple_"+model.__class__.__name__+"submission.csv"
#         submission.to_csv(a,index=False)



#----------------------------------------------------------------------

models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc,gbc,xgbc,lgbc,cat]


def pred_on_full_data(Xtrain,ytrain,ytrain2,Xtest,models):
    for model in models:
        print(model.__class__.__name__)
        model.fit(Xtrain, ytrain)
        y_test_pred_breed = model.predict(Xtest)
        model.fit(Xtrain, ytrain2)
        y_test_pred2_pet=model.predict(Xtest)
        id=pd.Series(test["pet_id"])
        submission=pd.DataFrame({'pet_id':id,'breed_category':y_test_pred_breed,'pet_category':y_test_pred2_pet})
#         predictions=pd.concat([test['INCIDENT_ID'],pd.DataFrame(y_test_pred,columns=['MULTIPLE_OFFENSE'])],1)
        a="submission_multiple_"+model.__class__.__name__+"submission.csv"
        submission.to_csv(a,index=False)
        

#getting predictions on full data
pred_on_full_data(Xtrain=train_full,ytrain=train_full_breed,ytrain2=train_full_pet,Xtest=test_full,models=models)

CatBoostClassifier
Learning rate set to 0.091906
0:	learn: 0.9534629	total: 80.9ms	remaining: 1m 20s
1:	learn: 0.8425060	total: 135ms	remaining: 1m 7s
2:	learn: 0.7538103	total: 188ms	remaining: 1m 2s
3:	learn: 0.6809594	total: 252ms	remaining: 1m 2s
4:	learn: 0.6197896	total: 294ms	remaining: 58.4s
5:	learn: 0.5673674	total: 335ms	remaining: 55.5s
6:	learn: 0.5223623	total: 375ms	remaining: 53.3s
7:	learn: 0.4833295	total: 453ms	remaining: 56.2s
8:	learn: 0.4493457	total: 495ms	remaining: 54.5s
9:	learn: 0.4190976	total: 528ms	remaining: 52.3s
10:	learn: 0.3924703	total: 558ms	remaining: 50.1s
11:	learn: 0.3689511	total: 598ms	remaining: 49.2s
12:	learn: 0.3482353	total: 639ms	remaining: 48.5s
13:	learn: 0.3296436	total: 702ms	remaining: 49.4s
14:	learn: 0.3131712	total: 756ms	remaining: 49.6s
15:	learn: 0.2982555	total: 829ms	remaining: 51s
16:	learn: 0.2847745	total: 869ms	remaining: 50.3s
17:	learn: 0.2726627	total: 925ms	remaining: 50.5s
18:	learn: 0.2617436	total: 1s	remaining: 5

161:	learn: 0.1427426	total: 5.68s	remaining: 29.4s
162:	learn: 0.1427039	total: 5.7s	remaining: 29.3s
163:	learn: 0.1426519	total: 5.72s	remaining: 29.2s
164:	learn: 0.1425457	total: 5.74s	remaining: 29.1s
165:	learn: 0.1425039	total: 5.75s	remaining: 28.9s
166:	learn: 0.1423985	total: 5.77s	remaining: 28.8s
167:	learn: 0.1421454	total: 5.79s	remaining: 28.7s
168:	learn: 0.1421116	total: 5.8s	remaining: 28.5s
169:	learn: 0.1420049	total: 5.82s	remaining: 28.4s
170:	learn: 0.1419445	total: 5.83s	remaining: 28.3s
171:	learn: 0.1418460	total: 5.85s	remaining: 28.2s
172:	learn: 0.1417357	total: 5.87s	remaining: 28.1s
173:	learn: 0.1416652	total: 5.89s	remaining: 28s
174:	learn: 0.1415932	total: 5.91s	remaining: 27.9s
175:	learn: 0.1415310	total: 5.92s	remaining: 27.7s
176:	learn: 0.1414736	total: 5.94s	remaining: 27.6s
177:	learn: 0.1414155	total: 5.96s	remaining: 27.5s
178:	learn: 0.1413429	total: 5.97s	remaining: 27.4s
179:	learn: 0.1412772	total: 5.99s	remaining: 27.3s
180:	learn: 0.14

323:	learn: 0.1313307	total: 8.04s	remaining: 16.8s
324:	learn: 0.1312700	total: 8.06s	remaining: 16.7s
325:	learn: 0.1311786	total: 8.07s	remaining: 16.7s
326:	learn: 0.1311161	total: 8.09s	remaining: 16.6s
327:	learn: 0.1310489	total: 8.1s	remaining: 16.6s
328:	learn: 0.1309242	total: 8.11s	remaining: 16.5s
329:	learn: 0.1308411	total: 8.13s	remaining: 16.5s
330:	learn: 0.1308019	total: 8.15s	remaining: 16.5s
331:	learn: 0.1307041	total: 8.16s	remaining: 16.4s
332:	learn: 0.1306563	total: 8.18s	remaining: 16.4s
333:	learn: 0.1305921	total: 8.19s	remaining: 16.3s
334:	learn: 0.1305256	total: 8.2s	remaining: 16.3s
335:	learn: 0.1304639	total: 8.22s	remaining: 16.2s
336:	learn: 0.1303935	total: 8.23s	remaining: 16.2s
337:	learn: 0.1303426	total: 8.25s	remaining: 16.2s
338:	learn: 0.1302876	total: 8.27s	remaining: 16.1s
339:	learn: 0.1302647	total: 8.29s	remaining: 16.1s
340:	learn: 0.1301583	total: 8.3s	remaining: 16s
341:	learn: 0.1301119	total: 8.31s	remaining: 16s
342:	learn: 0.13005

487:	learn: 0.1224932	total: 10.7s	remaining: 11.3s
488:	learn: 0.1224403	total: 10.7s	remaining: 11.2s
489:	learn: 0.1223919	total: 10.8s	remaining: 11.2s
490:	learn: 0.1223754	total: 10.8s	remaining: 11.2s
491:	learn: 0.1223198	total: 10.8s	remaining: 11.1s
492:	learn: 0.1222315	total: 10.8s	remaining: 11.1s
493:	learn: 0.1222066	total: 10.8s	remaining: 11.1s
494:	learn: 0.1221799	total: 10.8s	remaining: 11s
495:	learn: 0.1221340	total: 10.8s	remaining: 11s
496:	learn: 0.1220620	total: 10.8s	remaining: 11s
497:	learn: 0.1219927	total: 10.9s	remaining: 11s
498:	learn: 0.1219683	total: 10.9s	remaining: 10.9s
499:	learn: 0.1218947	total: 10.9s	remaining: 10.9s
500:	learn: 0.1218336	total: 10.9s	remaining: 10.9s
501:	learn: 0.1218065	total: 10.9s	remaining: 10.8s
502:	learn: 0.1217584	total: 10.9s	remaining: 10.8s
503:	learn: 0.1217121	total: 10.9s	remaining: 10.8s
504:	learn: 0.1216941	total: 11s	remaining: 10.7s
505:	learn: 0.1216468	total: 11s	remaining: 10.7s
506:	learn: 0.1216120	to

655:	learn: 0.1153543	total: 12.9s	remaining: 6.76s
656:	learn: 0.1153120	total: 12.9s	remaining: 6.74s
657:	learn: 0.1152837	total: 12.9s	remaining: 6.72s
658:	learn: 0.1152533	total: 12.9s	remaining: 6.69s
659:	learn: 0.1151766	total: 12.9s	remaining: 6.67s
660:	learn: 0.1151419	total: 13s	remaining: 6.65s
661:	learn: 0.1150844	total: 13s	remaining: 6.62s
662:	learn: 0.1150522	total: 13s	remaining: 6.6s
663:	learn: 0.1149828	total: 13s	remaining: 6.57s
664:	learn: 0.1149498	total: 13s	remaining: 6.55s
665:	learn: 0.1148964	total: 13s	remaining: 6.53s
666:	learn: 0.1148595	total: 13s	remaining: 6.5s
667:	learn: 0.1148112	total: 13s	remaining: 6.48s
668:	learn: 0.1147588	total: 13s	remaining: 6.46s
669:	learn: 0.1147443	total: 13.1s	remaining: 6.43s
670:	learn: 0.1147059	total: 13.1s	remaining: 6.41s
671:	learn: 0.1146420	total: 13.1s	remaining: 6.38s
672:	learn: 0.1145790	total: 13.1s	remaining: 6.36s
673:	learn: 0.1145648	total: 13.1s	remaining: 6.34s
674:	learn: 0.1144968	total: 13.

...

316:	learn: 0.2380502	total: 9.74s	remaining: 21s
317:	learn: 0.2379799	total: 9.75s	remaining: 20.9s
318:	learn: 0.2378248	total: 9.76s	remaining: 20.8s
319:	learn: 0.2377494	total: 9.78s	remaining: 20.8s
320:	learn: 0.2375814	total: 9.79s	remaining: 20.7s
321:	learn: 0.2374416	total: 9.8s	remaining: 20.6s
322:	learn: 0.2373241	total: 9.82s	remaining: 20.6s
323:	learn: 0.2372274	total: 9.83s	remaining: 20.5s
324:	learn: 0.2370631	total: 9.85s	remaining: 20.4s
325:	learn: 0.2369627	total: 9.86s	remaining: 20.4s
326:	learn: 0.2368319	total: 9.87s	remaining: 20.3s
327:	learn: 0.2367331	total: 9.89s	remaining: 20.3s
328:	learn: 0.2365650	total: 9.9s	remaining: 20.2s
329:	learn: 0.2363869	total: 9.91s	remaining: 20.1s
330:	learn: 0.2362084	total: 9.94s	remaining: 20.1s
331:	learn: 0.2361158	total: 9.95s	remaining: 20s
332:	learn: 0.2359663	total: 9.96s	remaining: 20s
333:	learn: 0.2358278	total: 9.98s	remaining: 19.9s
334:	learn: 0.2357935	total: 9.99s	remaining: 19.8s
335:	learn: 0.235697

481:	learn: 0.2179375	total: 12.1s	remaining: 13s
482:	learn: 0.2178700	total: 12.1s	remaining: 13s
483:	learn: 0.2177897	total: 12.1s	remaining: 12.9s
484:	learn: 0.2177478	total: 12.2s	remaining: 12.9s
485:	learn: 0.2177037	total: 12.2s	remaining: 12.9s
486:	learn: 0.2175738	total: 12.2s	remaining: 12.9s
487:	learn: 0.2171327	total: 12.2s	remaining: 12.8s
488:	learn: 0.2170334	total: 12.2s	remaining: 12.8s
489:	learn: 0.2169036	total: 12.3s	remaining: 12.8s
490:	learn: 0.2168226	total: 12.3s	remaining: 12.7s
491:	learn: 0.2167080	total: 12.3s	remaining: 12.7s
492:	learn: 0.2166212	total: 12.3s	remaining: 12.7s
493:	learn: 0.2164352	total: 12.3s	remaining: 12.6s
494:	learn: 0.2163016	total: 12.4s	remaining: 12.6s
495:	learn: 0.2162145	total: 12.4s	remaining: 12.6s
496:	learn: 0.2161373	total: 12.4s	remaining: 12.5s
497:	learn: 0.2159856	total: 12.4s	remaining: 12.5s
498:	learn: 0.2159086	total: 12.4s	remaining: 12.5s
499:	learn: 0.2154000	total: 12.4s	remaining: 12.4s
500:	learn: 0.21

651:	learn: 0.2001128	total: 14.7s	remaining: 7.83s
652:	learn: 0.1999787	total: 14.7s	remaining: 7.8s
653:	learn: 0.1998911	total: 14.7s	remaining: 7.78s
654:	learn: 0.1998216	total: 14.7s	remaining: 7.75s
655:	learn: 0.1997399	total: 14.7s	remaining: 7.72s
656:	learn: 0.1996476	total: 14.7s	remaining: 7.7s
657:	learn: 0.1995591	total: 14.8s	remaining: 7.67s
658:	learn: 0.1994972	total: 14.8s	remaining: 7.64s
659:	learn: 0.1994343	total: 14.8s	remaining: 7.61s
660:	learn: 0.1993483	total: 14.8s	remaining: 7.59s
661:	learn: 0.1992580	total: 14.8s	remaining: 7.56s
662:	learn: 0.1991388	total: 14.8s	remaining: 7.53s
663:	learn: 0.1989956	total: 14.8s	remaining: 7.51s
664:	learn: 0.1988850	total: 14.9s	remaining: 7.48s
665:	learn: 0.1988063	total: 14.9s	remaining: 7.45s
666:	learn: 0.1987448	total: 14.9s	remaining: 7.43s
667:	learn: 0.1986950	total: 14.9s	remaining: 7.4s
668:	learn: 0.1986324	total: 14.9s	remaining: 7.38s
669:	learn: 0.1985806	total: 14.9s	remaining: 7.35s
670:	learn: 0.1

813:	learn: 0.1862335	total: 17.1s	remaining: 3.9s
814:	learn: 0.1861176	total: 17.1s	remaining: 3.87s
815:	learn: 0.1859869	total: 17.1s	remaining: 3.85s
816:	learn: 0.1859462	total: 17.1s	remaining: 3.83s
817:	learn: 0.1858883	total: 17.1s	remaining: 3.81s
818:	learn: 0.1858399	total: 17.1s	remaining: 3.79s
819:	learn: 0.1857454	total: 17.1s	remaining: 3.76s
820:	learn: 0.1857061	total: 17.2s	remaining: 3.74s
821:	learn: 0.1856013	total: 17.2s	remaining: 3.72s
822:	learn: 0.1855469	total: 17.2s	remaining: 3.7s
823:	learn: 0.1854560	total: 17.2s	remaining: 3.68s
824:	learn: 0.1853718	total: 17.2s	remaining: 3.66s
825:	learn: 0.1853077	total: 17.3s	remaining: 3.64s
826:	learn: 0.1852155	total: 17.3s	remaining: 3.62s
827:	learn: 0.1851710	total: 17.3s	remaining: 3.6s
828:	learn: 0.1849219	total: 17.4s	remaining: 3.58s
829:	learn: 0.1848600	total: 17.4s	remaining: 3.56s
830:	learn: 0.1847898	total: 17.4s	remaining: 3.54s
831:	learn: 0.1847481	total: 17.4s	remaining: 3.52s
832:	learn: 0.1

977:	learn: 0.1741704	total: 20.8s	remaining: 468ms
978:	learn: 0.1741193	total: 20.8s	remaining: 446ms
979:	learn: 0.1740270	total: 20.8s	remaining: 425ms
980:	learn: 0.1739350	total: 20.8s	remaining: 404ms
981:	learn: 0.1738519	total: 20.9s	remaining: 382ms
982:	learn: 0.1738106	total: 20.9s	remaining: 361ms
983:	learn: 0.1737656	total: 20.9s	remaining: 340ms
984:	learn: 0.1737298	total: 20.9s	remaining: 319ms
985:	learn: 0.1736760	total: 20.9s	remaining: 297ms
986:	learn: 0.1736141	total: 21s	remaining: 276ms
987:	learn: 0.1735389	total: 21s	remaining: 255ms
988:	learn: 0.1735030	total: 21s	remaining: 234ms
989:	learn: 0.1734024	total: 21s	remaining: 212ms
990:	learn: 0.1733181	total: 21.1s	remaining: 191ms
991:	learn: 0.1732638	total: 21.1s	remaining: 170ms
992:	learn: 0.1732154	total: 21.1s	remaining: 149ms
993:	learn: 0.1731171	total: 21.1s	remaining: 127ms
994:	learn: 0.1729975	total: 21.1s	remaining: 106ms
995:	learn: 0.1729656	total: 21.2s	remaining: 85ms
996:	learn: 0.1728503